# RAMDOM FOREST PRUEBA

#### **La diferencia entre "Random Forest" y "Random Tree"**.

 Ambos son técnicas basadas en árboles de decisión, pero tienen propiedades y usos distintos:

Random Tree (Árbol Aleatorio):

Es un único árbol de decisión que se construye de manera aleatoria.
Durante la construcción del árbol, al momento de dividir un nodo, se selecciona un subconjunto aleatorio de características en lugar de considerar todas las características disponibles. Esto le da el carácter "aleatorio" al árbol.
El árbol aleatorio puede ser propenso al sobreajuste si se le permite crecer sin restricciones, especialmente si el conjunto de datos tiene muchas características o es ruidoso.
Es más simple y rápido de construir que un bosque aleatorio.
Random Forest (Bosque Aleatorio):

Es un conjunto (ensamblado) de árboles de decisión, comúnmente árboles aleatorios.
Durante el entrenamiento, cada árbol se construye usando un subconjunto aleatorio de los datos (esto se llama "bootstrap" de los datos). Además, al igual que en el árbol aleatorio, durante la construcción de cada árbol, se selecciona un subconjunto aleatorio de características en cada división.
El propósito de usar múltiples árboles es reducir la varianza y el riesgo de sobreajuste. Las predicciones del bosque aleatorio se obtienen promediando las predicciones (en problemas de regresión) o haciendo una votación (en problemas de clasificación) de todos los árboles en el ensamblado.
Es más robusto y preciso en la mayoría de los casos en comparación con un solo árbol aleatorio, debido al efecto de ensamblado.
En resumen, mientras que un "Random Tree" es un solo árbol que toma decisiones basadas en subconjuntos aleatorios de características, "Random Forest" es un conjunto de tales árboles que trabajan juntos para dar una predicción final. El enfoque de ensamblado de Random Forest le permite aprovechar la "sabiduría de la multitud", donde la combinación de muchos modelos ayuda a mejorar la precisión y reducir el sobreajuste.

In [1]:
import pandas as pd

#### cargar datos

In [ ]:
sample_file_path = '../dataset/data/sample.csv'
sample = pd.read_csv(sample_file_path) 

sample.columns
#sample.info()

In [3]:
sample_cod = sample.drop(['Attack', 'Dataset'], axis=1)

#### Codificacion one-hot ataques Solo ataques DDoS (archive sample_cod-15000 registros)

In [ ]:
encoded_attacks = pd.get_dummies(sample['Attack'], prefix='Attack')
encoded_attacks.reset_index(drop=True, inplace=True)
sample.reset_index(drop=True, inplace=True)
sample_cod['Attack_DDoS'] = encoded_attacks['Attack_DDoS'].astype('int64')
sample_cod.info()

In [6]:
sample_cod.to_csv('../dataset/data/sample_cod.csv', index=False)

#### lectura archivo csv

In [10]:
sample_cod = pd.read_csv('../dataset/data/sample_150K_cod.csv')

In [11]:
sample_cod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151927 entries, 0 to 151926
Data columns (total 44 columns):
 #   Column                       Non-Null Count   Dtype
---  ------                       --------------   -----
 0   L4_SRC_PORT                  151927 non-null  int64
 1   L4_DST_PORT                  151927 non-null  int64
 2   PROTOCOL                     151927 non-null  int64
 3   L7_PROTO                     151927 non-null  int64
 4   IN_BYTES                     151927 non-null  int64
 5   IN_PKTS                      151927 non-null  int64
 6   OUT_BYTES                    151927 non-null  int64
 7   OUT_PKTS                     151927 non-null  int64
 8   TCP_FLAGS                    151927 non-null  int64
 9   CLIENT_TCP_FLAGS             151927 non-null  int64
 10  SERVER_TCP_FLAGS             151927 non-null  int64
 11  FLOW_DURATION_MILLISECONDS   151927 non-null  int64
 12  DURATION_IN                  151927 non-null  int64
 13  DURATION_OUT                 

In [7]:
#y = sample_cod.Attack_DDoS
y = sample_cod.AttackCod

#### Seleccion de caracteristicas

In [8]:
#NIDS_features = ['L7_PROTO', 'FLOW_DURATION_MILLISECONDS', 'RETRANSMITTED_IN_PKTS', 'NUM_PKTS_128_TO_256_BYTES', 'MAX_IP_PKT_LEN']
NIDS_features = ['LONGEST_FLOW_PKT','TCP_FLAGS','CLIENT_TCP_FLAGS','MAX_IP_PKT_LEN', 'IN_BYTES', 'L7_PROTO', 'TCP_WIN_MAX_IN', 'NUM_PKTS_UP_TO_128_BYTES',
                 'MIN_TTL', 'MAX_TTL', 'NUM_PKTS_128_TO_256_BYTES','MIN_IP_PKT_LEN', 
                'DNS_QUERY_ID','DURATION_IN', 'FLOW_DURATION_MILLISECONDS', 'RETRANSMITTED_IN_PKTS']

In [9]:
X = sample_cod[NIDS_features]

In [10]:
X.describe()

,LONGEST_FLOW_PKT,TCP_FLAGS,CLIENT_TCP_FLAGS,MAX_IP_PKT_LEN,IN_BYTES,L7_PROTO,TCP_WIN_MAX_IN,NUM_PKTS_UP_TO_128_BYTES,MIN_TTL,MAX_TTL,NUM_PKTS_128_TO_256_BYTES,MIN_IP_PKT_LEN,DNS_QUERY_ID,DURATION_IN,FLOW_DURATION_MILLISECONDS,RETRANSMITTED_IN_PKTS
count,151927.000000,151927.000000,151927.000000,151927.000000,1.519270e+05,151927.000000,151927.000000,151927.000000,151927.000000,151927.000000,151927.000000,151927.000000,151927.000000,151927.000000,1.519270e+05,151927.000000
mean,264.269425,24.044199,22.041072,264.269425,1.013931e+03,53.365189,6022.475906,26.852916,53.639834,53.694169,0.976713,23.609688,4526.881101,534.016251,2.316412e+06,0.386185
std,425.507647,58.185309,58.255988,425.507647,8.601767e+04,78.920631,13205.205996,1470.859328,39.620603,39.658071,13.132424,26.990729,13321.616592,831.346125,2.140299e+06,2.969931
min,28.000000,0.000000,0.000000,28.000000,4.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
25%,40.000000,0.000000,0.000000,40.000000,5.600000e+01,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000
50%,100.000000,2.000000,2.000000,100.000000,1.120000e+02,7.000000,512.000000,2.000000,64.000000,64.000000,0.000000,0.000000,0.000000,21.000000,4.293061e+06,0.000000
75%,140.000000,22.000000,6.000000,140.000000,2.800000e+02,91.000000,4096.000000,4.000000,64.000000,64.000000,2.000000,40.000000,0.000000,1050.000000,4.294029e+06,0.000000
max,7292.000000,223.000000,223.000000,7292.000000,3.148497e+07,245.000000,65535.000000,183859.000000,255.000000,255.000000,3314.000000,422.000000,65534.000000,102900.000000,4.294966e+06,552.000000


#### Entrenamiento del modelo decisionTree

In [18]:
from sklearn.tree import DecisionTreeRegressor

# Define model. Specify a number for random_state to ensure same results each run
security_model = DecisionTreeRegressor(random_state=1)

# Fit model
security_model.fit(X, y)

DecisionTreeRegressor(random_state=1)

#### Predicciones con el modelo

In [19]:
print("Making predictions for the following 5 attacks:")
print(X.head())
print("The predictions are")
print(security_model.predict(X.head()))

Making predictions for the following 5 attacks:
   LONGEST_FLOW_PKT  TCP_FLAGS  CLIENT_TCP_FLAGS  MAX_IP_PKT_LEN  IN_BYTES  \
0               100          0                 0             100       200   
1               100          0                 0             100       200   
2               478         27                27             478       828   
3               423         27                27             423       794   
4                48         22                 2              48      1440   

   L7_PROTO  TCP_WIN_MAX_IN  NUM_PKTS_UP_TO_128_BYTES  MIN_TTL  MAX_TTL  \
0         0               0                         2      254      254   
1         0               0                         2      254      254   
2         7           16383                        14       62       63   
3         7           16383                        14       62       63   
4       131            8192                        60      128      128   

   NUM_PKTS_128_TO_256_BYTES  MI

#### Error Absoluto de las predicciones del modelo

In [20]:
from sklearn.metrics import mean_absolute_error
predicted_attacks = security_model.predict(X)
mean_absolute_error(y, predicted_attacks)

0.019786173927580326

#### Precision

In [ ]:
from sklearn.metrics import accuracy_score

# Supongamos que tienes etiquetas verdaderas en y_true y predicciones en predicted_attacks
accuracy = accuracy_score(y_true, predicted_attacks)

print("Accuracy:", accuracy)

#### Division (entrenamiento y validacion)

In [13]:
from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)
# Define model
ciber_model = DecisionTreeRegressor()
# Fit model
ciber_model.fit(train_X, train_y)

# get predicted prices on validation data
val_predictions = ciber_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

0.03246308013241967


#### Entrenamiento modelo 2 variando profundidad para encontrar punto optimo( sweet spot)

In [15]:
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor

def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

In [16]:
# compare MAE with differing values of max_leaf_nodes
for max_leaf_nodes in [5, 50, 500, 5000]:
    my_mae = get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

Max leaf nodes: 5  		 Mean Absolute Error:  0
Max leaf nodes: 50  		 Mean Absolute Error:  0
Max leaf nodes: 500  		 Mean Absolute Error:  0
Max leaf nodes: 5000  		 Mean Absolute Error:  0


#### Random Forests

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
attacks_preds = forest_model.predict(val_X)
print(mean_absolute_error(val_y, attacks_preds))

0.03295540802889015


#### RFE Eliminación de características Recursiva (RFE)

##### Explicacion

La Eliminación de Características Recursiva (RFE, por sus siglas en inglés, Recursive Feature Elimination) es una técnica de selección de características utilizada en aprendizaje automático para mejorar la eficiencia y el rendimiento de un modelo al eliminar de manera iterativa las características menos importantes del conjunto de datos. Su proceso se puede explicar en los siguientes pasos:

**Modelo Base**: Inicialmente, se entrena un modelo con todas las características disponibles en el conjunto de datos. El tipo de modelo utilizado depende del problema en cuestión, y puede ser una máquina de soporte vectorial (SVM), una regresión, una red neuronal, o cualquier otro algoritmo de aprendizaje automático.

**Ranking de Características**: Después de entrenar el modelo base, se obtiene una puntuación o ranking de importancia para cada característica. Esta puntuación puede basarse en coeficientes, como los coeficientes de una regresión, la importancia de las características en un árbol de decisión, o cualquier otra métrica relevante.

**Eliminación de Características**: Se elimina la característica menos importante según el ranking obtenido en el paso anterior. Esto significa que se descarta la característica que tiene la menor contribución al rendimiento del modelo.

**Entrenamiento Iterativo**: El modelo se vuelve a entrenar con el conjunto de datos reducido, que ahora contiene una característica menos. Este proceso se repite de manera iterativa, eliminando una característica en cada iteración, hasta que se alcanza un criterio de parada predefinido. Este criterio puede ser un número específico de características a mantener o una métrica de rendimiento del modelo.

**Evaluación del Rendimiento**: Se evalúa el rendimiento del modelo en un conjunto de validación o prueba en cada iteración. Esto permite determinar si la eliminación de características ha mejorado o degradado el rendimiento del modelo.

**Selección de Características Óptimas**: Finalmente, se selecciona el conjunto óptimo de características que proporciona el mejor rendimiento en función del criterio de evaluación establecido.

La Eliminación de Características Recursiva es útil cuando se tiene un conjunto de datos con muchas características y se desea reducir la dimensionalidad para mejorar la eficiencia computacional y evitar la maldición de la dimensionalidad. Al eliminar características menos relevantes, se puede obtener un modelo más simple y generalizable. La elección de un criterio de parada adecuado y la elección del modelo base son aspectos importantes en la implementación de RFE. La biblioteca scikit-learn de Python ofrece una implementación de RFE que facilita su aplicación en proyectos de aprendizaje automático.